## Plot Forecasts

In [27]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [28]:
import sys
sys.path.insert(0, '../')
import pandas as pd

## New Forecast Plots

In [29]:
approachB_fc = pd.read_csv(f'../combined_nn_results/refined_models/approachB/grid.csv')

In [30]:
approachB_fc

,date_str,power,fc
0,2021-01-09 07:00:00,11.49,2647.242676
1,2021-01-09 08:00:00,2827.62,4530.391113
2,2021-01-09 09:00:00,19126.00,37447.898438
3,2021-01-09 10:00:00,81339.20,78237.976562
4,2021-01-09 11:00:00,120248.86,114792.812500
...,...,...,...
499,2021-02-13 16:00:00,255891.15,278899.281250
500,2021-02-13 17:00:00,231299.38,270949.687500
501,2021-02-13 18:00:00,255489.24,255555.750000
502,2021-02-13 19:00:00,191863.39,189611.546875


In [31]:
approachB_fc = approachB_fc.set_index('date_str')

In [22]:
approachB_fc.index = pd.to_datetime(approachB_fc.index)

In [23]:
approachB_fc = approachB_fc.resample('1H').mean()

In [24]:
approachB_fc = approachB_fc.fillna(0)

In [26]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=approachB_fc.index, y = approachB_fc['power'], name = 'Target Generation'))
fig.add_trace(go.Scatter(x=approachB_fc.index, y = approachB_fc['fc'], name = 'Forecasts - Approach B'))

## Old Forecast Plots

In [3]:
def read_fc(model_path, ts):
    results = pd.read_csv(f'../{model_path}/final_results/{ts}.csv', index_col=[0])
    benchmark = (model_path.split('/')[0] == 'benchmark_results')
    if benchmark:
        return results['power'], results['average_fc']
    else:
        return results['power'], results['average_fc'], results['fc_1'], results['fc_2']

In [19]:
target, cnn_fc_layer_6_lr, cnn_fc1_layer_6_lr, cnn_fc2_layer_6_lr = read_fc('cnn_results/tcn2/layers_6/lr', 'grid')

In [18]:
target, cnn_fc_layer_6, cnn_fc1_layer_6, cnn_fc2_layer_6 = read_fc('cnn_results/tcn2/layers_6/lr', 'grid')

In [29]:
# target, cnn_fc, cnn_fc1, cnn_fc2 = read_fc('cnn_results/tcn', 'grid')
target, cnn_fc, cnn_fc1, cnn_fc2 = read_fc('cnn_results/tcn_new/dilation_2', 'grid')

In [30]:
target, lstm_fc, lstm_fc1, lstm_fc2 = read_fc('lstm_results', 'grid')

In [13]:
naive_power, naive_fc = read_fc('benchmark_results/naive', 'grid')

In [14]:
arima_power, arima_fc = read_fc('benchmark_results/arima', 'grid')

In [15]:
tbats_power, tbats_fc = read_fc('benchmark_results/tbats', 'grid')

In [20]:
def plot_fc(target, fc_array, models):
    names = ['average', 'fc1', 'fc2']
    benchmark = ['naive', 'arima', 'tbats']
    fig = go.Figure()

    fig.add_trace(go.Scatter(x = target.index, y = target.values, name ='target', marker = dict(color ='black' )))
    
    count = 0
    model_count = 0
    for fc in fc_array:
        if models[model_count] not in benchmark:
            if count%3 == 0 and count!=0:
                count = 0
                model_count +=1
        else:
#             print(model_count)
            model_count +=1
                
        if models[model_count] not in benchmark:
            fig.add_trace(go.Scatter(x = target.index, y = fc.values, name =f'{models[model_count]}_{names[count]}'))
        else:
            fig.add_trace(go.Scatter(x = target.index, y = fc.values, name =f'{models[model_count]}'))
        count +=1
    fig.show()

In [21]:
plot_fc(target, [cnn_fc, cnn_fc1, cnn_fc2, cnn_fc_layer_6_lr, cnn_fc1_layer_6_lr, cnn_fc2_layer_6_lr,
                cnn_fc_layer_6, cnn_fc1_layer_6, cnn_fc2_layer_6, lstm_fc, lstm_fc1, lstm_fc2, naive_fc, arima_fc,
                tbats_fc],
       ['cnn', 'cnn-layer6-lr', 'cnn-layer6', 'lstm', 'naive', 'arima', 'tbats'])

In [44]:
def plot_forecasts(target, fc_array, models, colours):
    fig = go.Figure()
    for h in range(0, 14*5, 14):
        showLegend = False
        if h == 0:
            showLegend = True
        h_data = target[h : h+14]
        fig.add_trace(go.Scatter(x = h_data.index, y = h_data.values, name ='target',
                                 marker = dict(color ='black' ), showlegend = showLegend))
    
    model_count = 0
    for fc in fc_array:
        for h in range(0, 14*5, 14):
            showLegend = False
            if h == 0:
                showLegend = True
            h_data = fc[h : h+14]
            fig.add_trace(go.Scatter(x = h_data.index, y = h_data.values, name =models[model_count], 
                                     marker = dict(color =colours[model_count]), showlegend = showLegend))
        model_count +=1
    fig.update_yaxes(title= "Generation (kW)")
    fig.show()

In [45]:
plot_forecasts(target, [cnn_fc, lstm_fc], ['CNN', 'LSTM'], ['blue', 'red'])

In [42]:
target, lstm_fc, lstm_fc1, lstm_fc2 = read_fc('lstm_results', '6010')

In [43]:
plot_fc(target, [lstm_fc, lstm_fc1, lstm_fc2],
       ['lstm'])

In [44]:
target, lstm_fc, lstm_fc1, lstm_fc2 = read_fc('lstm_results', '2107373071')

In [45]:
plot_fc(target, [lstm_fc, lstm_fc1, lstm_fc2],
       ['lstm'])

## Loss Curves

In [7]:
def read_loss_curve(model_path, ts, iter_num):
    return pd.read_pickle(f'../{model_path}/training_loss_{ts}_iteration_{iter_num}')

In [8]:
loss_cnn = read_loss_curve('cnn_results/tcn', 'grid', 1)
loss_cnn_layer6 = read_loss_curve('cnn_results/tcn2/layers_6', 'grid', 1)
loss_cnn_layer6_lr = read_loss_curve('cnn_results/tcn2/layers_6/lr', 'grid', 1)
loss_lstm = read_loss_curve('lstm_results', 'grid', 1)

In [9]:
def plot_loss(loss_array, models):

    fig = go.Figure()
    
    i =0
    for loss in loss_array:
        fig.add_trace(go.Scatter(y= loss['loss'] , name = f'{models[i]}_train-loss' ))
        fig.add_trace(go.Scatter(y= loss['val_loss'], name = f'{models[i]}_val-loss'))
        i +=1
    fig.show()

In [10]:
plot_loss([loss_cnn, loss_cnn_layer6, loss_cnn_layer6_lr, loss_lstm], ['cnn', 'cnn-layer6', 'cnn-layer6-lr', 'lstm'])

In [11]:
loss_lstm_6010 = read_loss_curve('lstm_results', '6010', 1)

In [12]:
plot_loss([loss_lstm_6010], ['lstm-pc'])

In [13]:
loss_lstm_sub = read_loss_curve('lstm_results', 'substation1', 1)

In [14]:
plot_loss([loss_lstm_sub], ['lstm-substation'])

New CNN loss curves

In [15]:
loss_cnn_new = read_loss_curve('cnn_results/tcn_new/dilation_2', 'grid', 1)
loss_cnn_new_filter64 = read_loss_curve('cnn_results/tcn_new/filter_64', 'grid', 1)

In [16]:
plot_loss([loss_cnn_new, loss_cnn_new_filter64], ['grid-cnn', 'grid-filter64'])

In [17]:
loss_cnn_new = read_loss_curve('cnn_results/tcn_new/dilation_2', '6010', 1)
loss_cnn_new_filter64 = read_loss_curve('cnn_results/tcn_new/filter_64', '6010', 1)

In [18]:
plot_loss([loss_cnn_new, loss_cnn_new_filter64], ['6010-cnn', '6010-filter64'])

In [19]:
loss_cnn_new = read_loss_curve('cnn_results/tcn_new/dilation_2', '806187417', 3)
loss_cnn_new_filter64 = read_loss_curve('cnn_results/tcn_new/filter_64', '806187417', 3)
plot_loss([loss_cnn_new, loss_cnn_new_filter64], ['site-cnn', 'site-filter64'])

LSTM new loss curves

In [20]:
loss_lstm_new = read_loss_curve('lstm_results/lstm2', 'grid', 1)
loss_lstm = read_loss_curve('lstm_results', 'grid', 1)
plot_loss([loss_lstm_new, loss_lstm], ['lstm-new', 'lstm-old'])

In [23]:
loss_lstm_new = read_loss_curve('lstm_results/lstm2', 'tline_1', 1)
loss_lstm = read_loss_curve('lstm_results', 'tline_1', 1)
plot_loss([loss_lstm_new, loss_lstm], ['lstm-new', 'lstm-old'])

In [24]:
loss_lstm_new = read_loss_curve('lstm_results/lstm2', 'substation1', 1)
loss_lstm = read_loss_curve('lstm_results', 'substation1', 1)
plot_loss([loss_lstm_new, loss_lstm], ['lstm-new', 'lstm-old'])

In [25]:
loss_lstm_new = read_loss_curve('lstm_results/lstm2', '6010', 1)
loss_lstm = read_loss_curve('lstm_results', '6010', 1)
plot_loss([loss_lstm_new, loss_lstm], ['lstm-new', 'lstm-old'])

In [26]:
loss_lstm_new = read_loss_curve('lstm_results/lstm2', '806187417', 3)
loss_lstm = read_loss_curve('lstm_results', '806187417', 3)
plot_loss([loss_lstm_new, loss_lstm], ['lstm-new', 'lstm-old'])

In [27]:
import pickle5 as pickle
import os